# CHAID

The EPC data contains several categorical variables with a lot of values. In order to find suitable features which will retain the most information, three feature sets are explored;
* data driven
* domain driven
* exhaustive

The first approach, termed data driven, uses statistical methods to reduce the number of variables. As the variables containing textual descriptions of the property have been created free-hand, many contain a large number of unique values. In some cases, only recorded for one property. The data driven approach uses a single level Chi-square Automatic Interaction Detector (CHAID) to group the levels within each categorical variable into a smaller number of groups. CHAID groups values with a similar response rate or in this context Energy Efficiency Rating (EER). 

This script run CHAID and stores the results in a dictionary

In [31]:
import numpy as np
import pandas as pd
import datetime
import os
import glob
import json
from CHAID import Tree
import re

In [32]:
# set variables from config file
config_path = os.path.abspath('..')

with open(config_path + '/config-example.json', 'r') as f:
    config = json.load(f)

processing_path = config['DEFAULT']['processing_path']
epc_train_fname = config['DEFAULT']['epc_train_fname']
epc_test_fname = config['DEFAULT']['epc_test_fname']
epc_train_clean_fname = config['DEFAULT']['epc_train_clean_fname']
epc_chaid_fname = config['DEFAULT']['epc_chaid_fname']
epc_fname_suffix = config['DEFAULT']['epc_fname_suffix']

In [33]:
dtype_dict = {'INSPECTION_DATE':'str'}

epc_train = pd.read_csv(os.path.join(processing_path,epc_train_clean_fname + epc_fname_suffix),
                        header = 0,
                        delimiter = ',',
                        dtype = dtype_dict,
                        parse_dates = ['INSPECTION_DATE'])

C:\Users\KingRemy\AppData\Local\Temp\ipykernel_13720\425336681.py:3: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  epc_train = pd.read_csv(os.path.join(processing_path,epc_train_clean_fname + epc_fname_suffix),


In [34]:
#Get quantile boundafries
quantiles = epc_train['CURRENT_ENERGY_EFFICIENCY'].describe()
print(quantiles['25%'])
print(quantiles['75%'])

56.0
73.0


In [35]:
#Create a new new target field within the training data
min_eff = epc_train['CURRENT_ENERGY_EFFICIENCY'].min()
max_eff = epc_train['CURRENT_ENERGY_EFFICIENCY'].max()
epc_train['eff_flag'] = pd.cut(epc_train['CURRENT_ENERGY_EFFICIENCY'],
                               bins = [min_eff,56.0,73.0,max_eff],
                               labels = ['0','99','1'])

#Drop unwated '99' level of eff_flag and convert to integer
epc_train = epc_train[epc_train['eff_flag'].isin(['0','1'])]
epc_train['eff_flag'] = epc_train['eff_flag'].astype(int)

In [36]:
#Get the {0,1} sample size by taking the smallest class from the 0
#and 1 outcomes
sample_size = epc_train['eff_flag'].value_counts().min()


#Subsample positive and negative samples
neg_eff_flag = epc_train[epc_train['eff_flag'] == 0].sample(sample_size,random_state = 1234,axis = 0)

pos_eff_flag = epc_train[epc_train['eff_flag'] == 1].sample(sample_size,random_state = 1234,axis = 0)

#Concatenate
epc_chaid = pd.concat([neg_eff_flag, pos_eff_flag])
epc_chaid['eff_flag'].value_counts()

#Randomly shuffle epc_CHAID and reset the index
epc_chaid = epc_chaid.sample(frac = 1).reset_index(drop=True)

In [37]:
epc_chaid

LMK_KEY     region  \
0                       1343994469102015071612382137759368  Harrogate   
1                        782116299102012043022022591727008       Adur   
2                        429313659962010051709444002738000    Watford   
3                        182531372152011051819185398990455    Lincoln   
4        cf6600c95444fd16554d6642992474e706e608bf716b69...     Pendle   
...                                                    ...        ...   
1912829                   97158500732013101015004098968499  Allerdale   
1912830  be4ec06267c17a53c018ed19e7490684bd28fec0d71bd0...   Worthing   
1912831                  643321569002011061707573381799108  Mansfield   
1912832  1b6d21c782d0f780ce78b8c16c45eae682acabaac2624f...      Rugby   
1912833                 1444949079742016051812502142469888    Chorley   

         POSTCODE  BUILDING_REFERENCE_NUMBER CURRENT_ENERGY_RATING  \
0         HG1 1JG                 7185167378                     C   
1        BN43 5BZ                 9110297968                     C   
2        WD18 0FA                   75502768                     B   
3         LN2 5BQ                 4575283568                     F   
4         BB9 5DL                10000287911                     G   
...           ...                        ...                   ...   
1912829  CA14 2NU                  998466468                     E   
1912830  BN12 6NG                10001051261                     D   
1912831  NG21 0EQ                 1056767868                     F   
1912832  CV21 3TE                10000374168                     C   
1912833   PR7 7JX                 6703674478                     B   

         CURRENT_ENERGY_EFFICIENCY PROPERTY_TYPE     BUILT_FORM  \
0                               78          Flat       Detached   
1                               80          Flat    Mid-Terrace   
2                               88          Flat            NaN   
3                               37         House  Semi-Detached   
4                               19         House    End-Terrace   
...                            ...           ...            ...   
1912829                         51         House    Mid-Terrace   
1912830                         56         House  Semi-Detached   
1912831                         37         House       Detached   
1912832                         74         House    End-Terrace   
1912833                         89         House  Semi-Detached   

        INSPECTION_DATE           COUNTY  ... FLOOR_HEIGHT  \
0            2015-07-16  North Yorkshire  ...          NaN   
1            2012-04-30      West Sussex  ...          NaN   
2            2010-05-17    Hertfordshire  ...         2.40   
3            2011-05-18     Lincolnshire  ...         2.63   
4            2022-08-22       Lancashire  ...         2.80   
...                 ...              ...  ...          ...   
1912829      2013-10-10          Cumbria  ...          NaN   
1912830      2020-12-17      West Sussex  ...         2.25   
1912831      2011-06-10  Nottinghamshire  ...         2.41   
1912832      2021-02-03     Warwickshire  ...         2.36   
1912833      2016-05-18       Lancashire  ...          NaN   

         MECHANICAL_VENTILATION  inspection_year  \
0                           NaN             2015   
1                       natural             2012   
2                           NaN             2010   
3                       natural             2011   
4                       natural             2022   
...                         ...              ...   
1912829                 natural             2013   
1912830                 natural             2020   
1912831                 natural             2011   
1912832                 natural             2021   
1912833                     NaN             2016   

         floors_average_thermal_transmittance  \
0                                         0.2   
1                                         NaN   
2          

In [38]:
#Numeric
var_list_num = epc_chaid.select_dtypes(include= 'number').columns.tolist()
var_list_num.remove('CURRENT_ENERGY_EFFICIENCY')

#Categorical
var_list_cat = epc_chaid.select_dtypes(include= ['object','category']).columns.tolist()
var_list_cat.remove('LMK_KEY')
var_list_cat.remove('POSTCODE')
var_list_cat.remove('CURRENT_ENERGY_RATING')

### Creating a dictionary of CHAID scores

In [39]:
chaid_dict = {}
for var in var_list_cat:
    #Set the inputs and outputs
    #The imputs are given as a dictionary along with the type
    #The output must be of string type
    #I have assume all features are nominal, we can change the features dictionary to include the ordinal type
    features = {var:'nominal'}
    label = 'eff_flag'
    #Create the Tree
    chaid_dict[var] = {}
    tree = Tree.from_pandas_df(epc_chaid, i_variables = features, d_variable = label, alpha_merge = 0.0)
    #Loop through all the nodes and enter into a dictionary
    print(f'\n\n\nVariable: {var}')
    print(f'p-value: {tree.tree_store[0].split.p}')
    print(f'Chi2: {tree.tree_store[0].split.score}')
    for i in range(1, len(tree.tree_store)):
        count = tree.tree_store[i].members[0] + tree.tree_store[i].members[1]
        rate = tree.tree_store[i].members[1] / count
        print(f'\nNode {i}:\n\tCount = {count}\tRate = {rate}')
        print(f'\t{tree.tree_store[i].choices}')
        chaid_dict[var]['node' + str(i)] = tree.tree_store[i].choices




Variable: region
p-value: 0.0
Chi2: 65517.82722484344

Node 1:
	Count = 433464.0	Rate = 0.45513352896665005
	['Adur', 'Gedling', 'Bassetlaw', 'Cotswold', 'Teignbridge', 'Boston', 'Thanet', 'Hastings', 'Ashfield', 'Babergh', 'Lewes', 'Waverley', 'Breckland', 'Guildford', 'Worcester', 'Fylde', 'Wealden', 'Bolsover', 'Brentwood', 'Carlisle', 'Broadland', 'Rochford', 'Dover', 'Stroud', 'Chichester']

Node 2:
	Count = 301953.0	Rate = 0.37460796878984476
	['Allerdale', 'Rossendale', 'Craven', 'Broxtowe', 'Ryedale', 'Torridge', 'Copeland', 'Richmondshire', 'Erewash', 'Tendring', 'Worthing', 'Rother', 'Scarborough', 'Hambleton', 'Harrogate', 'Lancaster', 'Wyre', 'Melton', 'Sevenoaks', 'Maldon']

Node 3:
	Count = 364964.0	Rate = 0.5059348319286285
	['Arun', 'Blaby', 'Sedgemoor', 'Braintree', 'Swale', 'Havant', 'Stafford', 'Canterbury', 'Cheltenham', 'Preston', 'Bromsgrove', 'Spelthorne', 'Chesterfield', 'Fenland', 'Mansfield', 'Elmbridge', 'Mendip', 'Rushcliffe', 'Tandridge', 'Gravesham']

N

In [40]:
%store chaid_dict

Stored 'chaid_dict' (dict)
